# Generate Language models

In [1]:
from argparse import Namespace

import copy
import glob
import os
import pandas as pd
import pickle
import re
import spacy
from time import time

from gensim.models import Word2Vec
from gensim import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [2]:
#nlp = spacy.load("en_core_web_lg")

2019-11-12 11:40:34,896 : INFO : Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex .
2019-11-12 11:40:34,937 : INFO : Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex .


## Initialize a pre-trained language model

In [3]:
# model types
# w2v: word2vec
# ft: fasttext
model_type = "w2v"   
model_path = "/Users/khosseini/myJobs/ATI/Projects/2019/Living-with-Machines-code/language-lab-mro/lexicon_expansion/interactive_expansion/models/all_books/w2v_005/w2v_words.model"

if model_type.lower() in ["w2v", "word2vec"]:
    # Word2Vec
    embedding_model = Word2Vec.load(model_path)
elif model_type.lower() in ["ft", "fasttext"]:
    # FastText
    embedding_model = FastText.load(model_path)
print(embedding_model)
embedding_model_orig = copy.deepcopy(embedding_model)

2019-11-12 11:40:49,028 : INFO : loading Word2Vec object from /Users/khosseini/myJobs/ATI/Projects/2019/Living-with-Machines-code/language-lab-mro/lexicon_expansion/interactive_expansion/models/all_books/w2v_005/w2v_words.model
2019-11-12 11:40:51,244 : INFO : loading wv recursively from /Users/khosseini/myJobs/ATI/Projects/2019/Living-with-Machines-code/language-lab-mro/lexicon_expansion/interactive_expansion/models/all_books/w2v_005/w2v_words.model.wv.* with mmap=None
2019-11-12 11:40:51,252 : INFO : loading vectors from /Users/khosseini/myJobs/ATI/Projects/2019/Living-with-Machines-code/language-lab-mro/lexicon_expansion/interactive_expansion/models/all_books/w2v_005/w2v_words.model.wv.vectors.npy with mmap=None
2019-11-12 11:40:51,857 : INFO : setting ignored attribute vectors_norm to None
2019-11-12 11:40:51,862 : INFO : loading vocabulary recursively from /Users/khosseini/myJobs/ATI/Projects/2019/Living-with-Machines-code/language-lab-mro/lexicon_expansion/interactive_expansion/m

Word2Vec(vocab=434049, size=300, alpha=0.03)


## Read DB

In [4]:
db_sentence = pd.read_pickle("./db_trove_sentence_lookup.pkl")
db_sentence.head()

,filePath,articleId,articleType,year,ocrText,humanText,corrected,str_similarity,str_length_humanText,str_length_ocrText,quality_band,use_corrected,corrected_sentencizer,corrected_dict_lookup,ocr_sentencizer,ocr_dict_lookup
1,./trove_overproof/datasets/dataset1/rawTextAnd...,18378453,Article ILLUSTRATED,1953,"FROM RIVER CROSSING TO END OF TRIÄÜ I ^PI A^H""...",FROM RIVER CROSSING TO END OF TRIAL SPLASH: Pe...,FROM RIVER CROSSING TO END OF TRIAL SPLASH: Pe...,0.847561,746,820,2,0,"[[FROM, RIVER, CROSSING, TO, END, OF, TRIAL, S...","[[], [], [], [], [], [], [], []]","[[FROM, RIVER, CROSSING, TO, END, OF, TRIÄÜ, I...","[[], [], [], [], [], [], [], [], [], []]"
2,./trove_overproof/datasets/dataset1/rawTextAnd...,18363627,Article,1953,"Natural Childbirth Sir,-We nurses have seen fa...","Natural Childbirth Sir,-We nurses have seen fa...","Natural Childbirth Sir,-We nurses have seen fa...",0.964119,641,630,1,0,"[[Natural, Childbirth, Sir,-We, nurses, have, ...","[[], [], [], [], [], [], []]","[[Natural, Childbirth, Sir,-We, nurses, have, ...","[[], [], [], [], [], [], []]"
3,./trove_overproof/datasets/dataset1/rawTextAnd...,18366055,Article,1953,FIRST CHURCH I SERVICE 1 Presbyterian I ' Anni...,FIRST CHURCH SERVICE Presbyterian Anniversary ...,FIRST CHURCH SERVICE Presbyterian Anniversary ...,0.738901,946,832,3,0,"[[FIRST, CHURCH, SERVICE, Presbyterian, Annive...","[[], [], [], [], [], [], [], [], [], [], [], []]","[[FIRST, CHURCH, I, SERVICE, 1, Presbyterian, ...","[[], []]"
4,./trove_overproof/datasets/dataset1/rawTextAnd...,18386137,Article,1953,"""Bob"" Lulham's Fight Against Thallium District...","""Bob"" Lulham's Fight Against Thallium Arthur ...","""Bob"" Lulham's Fight Against Thallium Arthur ...",0.493898,2950,2740,4,0,"[["", Bob, "", Lulham, 's, Fight, Against, Thall...","[[], [], [], [], [], [], [], [], [], [], [], [...","[["", Bob, "", Lulham, 's, Fight, Against, Thall...","[[], [], [], [], [], [], [], [], [], [], [], [..."
5,./trove_overproof/datasets/dataset1/rawTextAnd...,18368961,Article,1953,"DIVORCE Before The Judge In Divorce, Mr Justic...","DIVORCE Before The Judge In Divorce, Mr. Justi...","DIVORCE Before The Judge In Divorce, Mr. Justi...",0.894176,1219,1121,2,0,"[[DIVORCE, Before, The, Judge, In, Divorce, ,,...","[[], [], [], [], [], [], [], []]","[[DIVORCE, Before, The, Judge, In, Divorce, ,,...","[[], []]"


## preprocess DB

In [ ]:
"""
def cleanup(myrow, colname="corrected"):
    # remove all # and @§
    
    corpus = [re.sub(r'#', '', element, flags=re.IGNORECASE) for element in corpus]
    corpus = [re.sub(r'@', '', element, flags=re.IGNORECASE) for element in corpus]
    
    # --- remove 2 or more .
    corpus = [re.sub('[.]{2,}', '.', element) for element in corpus]
    # --- add a space before and after a list of punctuations
    corpus = [re.sub(r"([.,!?:;\"\'])", r" \1 ", element) for element in corpus]
    # --- remove everything except:
    #corpus = [re.sub(r"([^a-zA-Z\-.:;,!?\d+]+)", r" ", element) for element in corpus]
    corpus = [re.sub(r"([^a-zA-Z\d+]+)", r" ", element) for element in corpus]
    # --- replace numbers with <NUM>
    corpus = [re.sub(r'\b\d+\b', '<NUM>', element) for element in corpus]
    corpus = [re.sub('--', '', element) for element in corpus]
    # --- normalize white spaces
    corpus = [re.sub('\s+', ' ', element) for element in corpus]
    
    # remove multiple spaces
    corpus = [re.sub(r'\s+', ' ', element, flags=re.IGNORECASE) for element in corpus]
    corpus = [element.strip() for element in corpus]
    #corpus = [element.lower() for element in corpus]
    return corpus
"""

In [5]:
def cleanup(myrow, col_name):
    all_clean_rows = []
    for sent in myrow[col_name]:
        one_clean_row = []
        for token in sent:
            one_clean_row.append(token.lower())
        all_clean_rows.append(one_clean_row)
    return all_clean_rows

In [6]:
db_sentence["ocr_sentencizer_cleaned"] = db_sentence.apply(cleanup, args=["ocr_sentencizer"], axis=1)
db_sentence["corrected_sentencizer_cleaned"] = db_sentence.apply(cleanup, args=["corrected_sentencizer"], axis=1)

In [7]:
db_sentence.head()

,filePath,articleId,articleType,year,ocrText,humanText,corrected,str_similarity,str_length_humanText,str_length_ocrText,quality_band,use_corrected,corrected_sentencizer,corrected_dict_lookup,ocr_sentencizer,ocr_dict_lookup,ocr_sentencizer_cleaned,corrected_sentencizer_cleaned
1,./trove_overproof/datasets/dataset1/rawTextAnd...,18378453,Article ILLUSTRATED,1953,"FROM RIVER CROSSING TO END OF TRIÄÜ I ^PI A^H""...",FROM RIVER CROSSING TO END OF TRIAL SPLASH: Pe...,FROM RIVER CROSSING TO END OF TRIAL SPLASH: Pe...,0.847561,746,820,2,0,"[[FROM, RIVER, CROSSING, TO, END, OF, TRIAL, S...","[[], [], [], [], [], [], [], []]","[[FROM, RIVER, CROSSING, TO, END, OF, TRIÄÜ, I...","[[], [], [], [], [], [], [], [], [], []]","[[from, river, crossing, to, end, of, triäü, i...","[[from, river, crossing, to, end, of, trial, s..."
2,./trove_overproof/datasets/dataset1/rawTextAnd...,18363627,Article,1953,"Natural Childbirth Sir,-We nurses have seen fa...","Natural Childbirth Sir,-We nurses have seen fa...","Natural Childbirth Sir,-We nurses have seen fa...",0.964119,641,630,1,0,"[[Natural, Childbirth, Sir,-We, nurses, have, ...","[[], [], [], [], [], [], []]","[[Natural, Childbirth, Sir,-We, nurses, have, ...","[[], [], [], [], [], [], []]","[[natural, childbirth, sir,-we, nurses, have, ...","[[natural, childbirth, sir,-we, nurses, have, ..."
3,./trove_overproof/datasets/dataset1/rawTextAnd...,18366055,Article,1953,FIRST CHURCH I SERVICE 1 Presbyterian I ' Anni...,FIRST CHURCH SERVICE Presbyterian Anniversary ...,FIRST CHURCH SERVICE Presbyterian Anniversary ...,0.738901,946,832,3,0,"[[FIRST, CHURCH, SERVICE, Presbyterian, Annive...","[[], [], [], [], [], [], [], [], [], [], [], []]","[[FIRST, CHURCH, I, SERVICE, 1, Presbyterian, ...","[[], []]","[[first, church, i, service, 1, presbyterian, ...","[[first, church, service, presbyterian, annive..."
4,./trove_overproof/datasets/dataset1/rawTextAnd...,18386137,Article,1953,"""Bob"" Lulham's Fight Against Thallium District...","""Bob"" Lulham's Fight Against Thallium Arthur ...","""Bob"" Lulham's Fight Against Thallium Arthur ...",0.493898,2950,2740,4,0,"[["", Bob, "", Lulham, 's, Fight, Against, Thall...","[[], [], [], [], [], [], [], [], [], [], [], [...","[["", Bob, "", Lulham, 's, Fight, Against, Thall...","[[], [], [], [], [], [], [], [], [], [], [], [...","[["", bob, "", lulham, 's, fight, against, thall...","[["", bob, "", lulham, 's, fight, against, thall..."
5,./trove_overproof/datasets/dataset1/rawTextAnd...,18368961,Article,1953,"DIVORCE Before The Judge In Divorce, Mr Justic...","DIVORCE Before The Judge In Divorce, Mr. Justi...","DIVORCE Before The Judge In Divorce, Mr. Justi...",0.894176,1219,1121,2,0,"[[DIVORCE, Before, The, Judge, In, Divorce, ,,...","[[], [], [], [], [], [], [], []]","[[DIVORCE, Before, The, Judge, In, Divorce, ,,...","[[], []]","[[divorce, before, the, judge, in, divorce, ,,...","[[divorce, before, the, judge, in, divorce, ,,..."


## Update a pre-trained LM

In [8]:
# args for Word2Vec
w2v_args = Namespace(
    epochs=5, 
    # only for Word2Vec
    compute_loss=True,                               # If True, computes and stores loss value which can be retrieved using get_latest_training_loss().

#     size=100,                                        # Dimensionality of the word vectors.
#     alpha=0.03,                                      # The initial learning rate.
#     min_alpha=0.0007,                                # Learning rate will linearly drop to min_alpha as training progresses.
#     sg=1,                                            # Training algorithm: skip-gram if sg=1, otherwise CBOW.
#     hs=0,                                            # If 1, hierarchical softmax will be used for model training. If set to 0, and negative is non-zero, negative sampling will be used.
#     negative=20,                                     # If > 0, negative sampling will be used, the int for negative specifies how many “noise words” should be drawn (usually between 5-20). If set to 0, no negative sampling is used. 
#     min_count=5,                                    # The model ignores all words with total frequency lower than this.
#     window=5,                                        # The maximum distance between the current and predicted word within a sentence.
#     sample=1e-3,                                     # The threshold for configuring which higher-frequency words are randomly downsampled, useful range is (0, 1e-5).
#     workers=8, 
#     cbow_mean=1,                                     # If 0, use the sum of the context word vectors. If 1, use the mean, only applies when cbow is used.
#     null_word=0,                                     # 
#     trim_rule=None,                                  # 
#     sorted_vocab=1,                                  # If 1, sort the vocabulary by descending frequency before assigning word indices.
#     batch_words=10000,                               # Target size (in words) for batches of examples passed to worker threads (and thus cython routines).(Larger batches will be passed if individual texts are longer than 10000 words, but the standard cython code truncates to that maximum.)
    
#     seed=1364,                                       # Seed for the random number generator.
#     # only for FastText (compare to word2vec)
#     #word_ngrams=1,                                   # If 1, uses enriches word vectors with subword(n-grams) information. If 0, this is equivalent to Word2Vec. 
#     #min_n=2,                                         # Minimum length of char n-grams to be used for training word representations.
#     #max_n=15,                                        # Max length of char ngrams to be used for training word representations. Set max_n to be lesser than min_n to avoid char ngrams being used.
#     #bucket=2000000                                  # Character ngrams are hashed into a fixed number of buckets, in order to limit the memory usage of the model. This option specifies the number of buckets used by the model.
)

In [ ]:
# Only if a new LM needs to be trained (from scratch)
"""
w2v_model = Word2Vec(
                 size=w2v_args.size, 
                 alpha=w2v_args.alpha,
                 min_alpha=w2v_args.min_alpha, 
                 sg=w2v_args.sg, 
                 hs=w2v_args.hs, 
                 negative=w2v_args.negative, 
                 iter=w2v_args.epochs, 
                 min_count=w2v_args.min_count, 
                 window=w2v_args.window, 
                 sample=w2v_args.sample, 
                 workers=w2v_args.workers, 
                 cbow_mean=w2v_args.cbow_mean, 
                 null_word=w2v_args.null_word, 
                 trim_rule=w2v_args.trim_rule, 
                 sorted_vocab=w2v_args.sorted_vocab, 
                 batch_words=w2v_args.batch_words, 
                 seed=w2v_args.seed, 
                 compute_loss=w2v_args.compute_loss)
"""

## Preprocess before creating/updating LM

In [ ]:
"""
def preprocess4LM(myrow, col_name="ocrText_cleaned_tokenize"):
    txt = [token.lemma_ for token in nlp(myrow[col_name].lower())]
    return txt
"""

In [ ]:
"""
db_sentence["ocrText_cleaned_tokenize"] = db_sentence[0:10].apply(preprocess4LM, args=["ocrText_cleaned"], axis=1)
db_sentence["corrected_cleaned_tokenize"] = db_sentence[0:10].apply(preprocess4LM, args=["corrected_cleaned"], axis=1)
"""

## OCR

In [9]:
list_sentences = db_sentence["ocr_sentencizer_cleaned"].to_list()
print('#sentences: {}'.format(len(list_sentences)))

#sentences: 30509


In [10]:
flattened_list_sentences = [val for sublist in list_sentences for val in sublist]

In [11]:
flattened_list_sentences[0]

['from',
 'river',
 'crossing',
 'to',
 'end',
 'of',
 'triäü',
 'i',
 '^',
 'pi',
 'a^h',
 '"',
 'pclcr',
 'antill',
 'ploughed',
 'deep',
 'into',
 'paddy',
 "'s",
 'river',
 'in',
 'his',
 'chrysler',
 'plymouth',
 'jr',
 'la',
 'jil',
 '?',
 'during',
 '{',
 '|',
 ')',
 'c',
 'elimination',
 'section',
 '.']

In [12]:
embedding_model_ocr = copy.deepcopy(embedding_model_orig)

In [13]:
embedding_model_ocr.workers = 8

In [14]:
embedding_model_ocr.build_vocab(flattened_list_sentences, update=True)
embedding_model_ocr.train(flattened_list_sentences, 
                          total_examples=embedding_model_ocr.corpus_count,
                          epochs=w2v_args.epochs,  
                          compute_loss=w2v_args.compute_loss)

2019-11-12 11:42:23,222 : INFO : collecting all words and their counts
2019-11-12 11:42:23,223 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2019-11-12 11:42:23,404 : INFO : PROGRESS: at sentence #10000, processed 525154 words, keeping 85319 word types
2019-11-12 11:42:23,526 : INFO : PROGRESS: at sentence #20000, processed 856174 words, keeping 117320 word types
2019-11-12 11:42:23,693 : INFO : PROGRESS: at sentence #30000, processed 1262312 words, keeping 157071 word types
2019-11-12 11:42:23,876 : INFO : PROGRESS: at sentence #40000, processed 1640602 words, keeping 191468 word types
2019-11-12 11:42:24,017 : INFO : PROGRESS: at sentence #50000, processed 1983065 words, keeping 219352 word types
2019-11-12 11:42:24,174 : INFO : PROGRESS: at sentence #60000, processed 2332908 words, keeping 244407 word types
2019-11-12 11:42:24,345 : INFO : PROGRESS: at sentence #70000, processed 2683098 words, keeping 272037 word types
2019-11-12 11:42:24,577 : INFO : PR

2019-11-12 11:43:12,892 : INFO : EPOCH 1 - PROGRESS: at 32.68% examples, 63715 words/s, in_qsize 15, out_qsize 0
2019-11-12 11:43:13,900 : INFO : EPOCH 1 - PROGRESS: at 33.56% examples, 64073 words/s, in_qsize 16, out_qsize 1
2019-11-12 11:43:15,151 : INFO : EPOCH 1 - PROGRESS: at 34.40% examples, 64027 words/s, in_qsize 15, out_qsize 0
2019-11-12 11:43:16,172 : INFO : EPOCH 1 - PROGRESS: at 35.38% examples, 64217 words/s, in_qsize 15, out_qsize 0
2019-11-12 11:43:17,289 : INFO : EPOCH 1 - PROGRESS: at 36.52% examples, 64374 words/s, in_qsize 15, out_qsize 0
2019-11-12 11:43:18,327 : INFO : EPOCH 1 - PROGRESS: at 37.10% examples, 64398 words/s, in_qsize 15, out_qsize 0
2019-11-12 11:43:19,417 : INFO : EPOCH 1 - PROGRESS: at 38.25% examples, 64732 words/s, in_qsize 15, out_qsize 0
2019-11-12 11:43:20,486 : INFO : EPOCH 1 - PROGRESS: at 39.26% examples, 64797 words/s, in_qsize 15, out_qsize 0
2019-11-12 11:43:21,655 : INFO : EPOCH 1 - PROGRESS: at 40.23% examples, 64420 words/s, in_qsize

2019-11-12 11:44:32,381 : INFO : EPOCH 1 - PROGRESS: at 95.06% examples, 64895 words/s, in_qsize 15, out_qsize 0
2019-11-12 11:44:33,542 : INFO : EPOCH 1 - PROGRESS: at 95.73% examples, 64982 words/s, in_qsize 15, out_qsize 0
2019-11-12 11:44:34,542 : INFO : EPOCH 1 - PROGRESS: at 96.33% examples, 64928 words/s, in_qsize 16, out_qsize 0
2019-11-12 11:44:35,599 : INFO : EPOCH 1 - PROGRESS: at 96.79% examples, 64909 words/s, in_qsize 15, out_qsize 0
2019-11-12 11:44:36,782 : INFO : EPOCH 1 - PROGRESS: at 97.17% examples, 64828 words/s, in_qsize 16, out_qsize 0
2019-11-12 11:44:37,798 : INFO : EPOCH 1 - PROGRESS: at 98.10% examples, 64824 words/s, in_qsize 15, out_qsize 0
2019-11-12 11:44:39,084 : INFO : EPOCH 1 - PROGRESS: at 98.97% examples, 64683 words/s, in_qsize 10, out_qsize 0
2019-11-12 11:44:39,185 : INFO : worker thread finished; awaiting finish of 7 more threads
2019-11-12 11:44:39,323 : INFO : worker thread finished; awaiting finish of 6 more threads
2019-11-12 11:44:39,375 : I

2019-11-12 11:45:45,810 : INFO : EPOCH 2 - PROGRESS: at 49.81% examples, 59827 words/s, in_qsize 15, out_qsize 0
2019-11-12 11:45:46,811 : INFO : EPOCH 2 - PROGRESS: at 50.30% examples, 59906 words/s, in_qsize 15, out_qsize 0
2019-11-12 11:45:47,961 : INFO : EPOCH 2 - PROGRESS: at 51.15% examples, 60054 words/s, in_qsize 15, out_qsize 0
2019-11-12 11:45:49,313 : INFO : EPOCH 2 - PROGRESS: at 51.94% examples, 59840 words/s, in_qsize 16, out_qsize 0
2019-11-12 11:45:50,391 : INFO : EPOCH 2 - PROGRESS: at 53.06% examples, 60266 words/s, in_qsize 15, out_qsize 0
2019-11-12 11:45:51,600 : INFO : EPOCH 2 - PROGRESS: at 53.87% examples, 60156 words/s, in_qsize 15, out_qsize 0
2019-11-12 11:45:52,644 : INFO : EPOCH 2 - PROGRESS: at 54.95% examples, 60527 words/s, in_qsize 16, out_qsize 0
2019-11-12 11:45:53,774 : INFO : EPOCH 2 - PROGRESS: at 55.75% examples, 60476 words/s, in_qsize 15, out_qsize 0
2019-11-12 11:45:54,777 : INFO : EPOCH 2 - PROGRESS: at 56.95% examples, 60702 words/s, in_qsize

2019-11-12 11:46:57,761 : INFO : EPOCH 3 - PROGRESS: at 5.04% examples, 55794 words/s, in_qsize 14, out_qsize 1
2019-11-12 11:46:59,558 : INFO : EPOCH 3 - PROGRESS: at 6.29% examples, 52831 words/s, in_qsize 15, out_qsize 2
2019-11-12 11:47:00,641 : INFO : EPOCH 3 - PROGRESS: at 7.20% examples, 54587 words/s, in_qsize 15, out_qsize 0
2019-11-12 11:47:02,115 : INFO : EPOCH 3 - PROGRESS: at 8.48% examples, 54323 words/s, in_qsize 15, out_qsize 0
2019-11-12 11:47:03,169 : INFO : EPOCH 3 - PROGRESS: at 9.33% examples, 55688 words/s, in_qsize 16, out_qsize 0
2019-11-12 11:47:04,657 : INFO : EPOCH 3 - PROGRESS: at 10.21% examples, 55301 words/s, in_qsize 15, out_qsize 0
2019-11-12 11:47:05,690 : INFO : EPOCH 3 - PROGRESS: at 11.15% examples, 56440 words/s, in_qsize 16, out_qsize 0
2019-11-12 11:47:06,939 : INFO : EPOCH 3 - PROGRESS: at 12.11% examples, 56834 words/s, in_qsize 15, out_qsize 0
2019-11-12 11:47:07,985 : INFO : EPOCH 3 - PROGRESS: at 13.36% examples, 57688 words/s, in_qsize 15, 

2019-11-12 11:48:20,331 : INFO : EPOCH 3 - PROGRESS: at 64.76% examples, 57867 words/s, in_qsize 15, out_qsize 0
2019-11-12 11:48:21,368 : INFO : EPOCH 3 - PROGRESS: at 65.19% examples, 57786 words/s, in_qsize 15, out_qsize 0
2019-11-12 11:48:22,404 : INFO : EPOCH 3 - PROGRESS: at 65.86% examples, 57775 words/s, in_qsize 15, out_qsize 0
2019-11-12 11:48:23,485 : INFO : EPOCH 3 - PROGRESS: at 66.41% examples, 57734 words/s, in_qsize 15, out_qsize 0
2019-11-12 11:48:24,545 : INFO : EPOCH 3 - PROGRESS: at 67.08% examples, 57701 words/s, in_qsize 16, out_qsize 0
2019-11-12 11:48:25,580 : INFO : EPOCH 3 - PROGRESS: at 67.79% examples, 57690 words/s, in_qsize 15, out_qsize 0
2019-11-12 11:48:26,605 : INFO : EPOCH 3 - PROGRESS: at 68.55% examples, 57682 words/s, in_qsize 16, out_qsize 0
2019-11-12 11:48:27,631 : INFO : EPOCH 3 - PROGRESS: at 69.50% examples, 57891 words/s, in_qsize 15, out_qsize 0
2019-11-12 11:48:28,711 : INFO : EPOCH 3 - PROGRESS: at 70.54% examples, 57983 words/s, in_qsize

2019-11-12 11:49:33,304 : INFO : EPOCH 4 - PROGRESS: at 28.42% examples, 69173 words/s, in_qsize 15, out_qsize 0
2019-11-12 11:49:34,514 : INFO : EPOCH 4 - PROGRESS: at 29.31% examples, 68785 words/s, in_qsize 16, out_qsize 0
2019-11-12 11:49:35,575 : INFO : EPOCH 4 - PROGRESS: at 30.25% examples, 68796 words/s, in_qsize 14, out_qsize 1
2019-11-12 11:49:36,803 : INFO : EPOCH 4 - PROGRESS: at 31.12% examples, 68788 words/s, in_qsize 15, out_qsize 0
2019-11-12 11:49:37,805 : INFO : EPOCH 4 - PROGRESS: at 31.82% examples, 68873 words/s, in_qsize 15, out_qsize 0
2019-11-12 11:49:38,894 : INFO : EPOCH 4 - PROGRESS: at 32.57% examples, 68733 words/s, in_qsize 15, out_qsize 0
2019-11-12 11:49:40,118 : INFO : EPOCH 4 - PROGRESS: at 33.29% examples, 68214 words/s, in_qsize 15, out_qsize 0
2019-11-12 11:49:41,141 : INFO : EPOCH 4 - PROGRESS: at 33.95% examples, 67935 words/s, in_qsize 16, out_qsize 0
2019-11-12 11:49:42,202 : INFO : EPOCH 4 - PROGRESS: at 35.01% examples, 68465 words/s, in_qsize

2019-11-12 11:50:52,727 : INFO : EPOCH 4 - PROGRESS: at 90.57% examples, 66328 words/s, in_qsize 15, out_qsize 0
2019-11-12 11:50:53,879 : INFO : EPOCH 4 - PROGRESS: at 91.59% examples, 66281 words/s, in_qsize 16, out_qsize 0
2019-11-12 11:50:54,938 : INFO : EPOCH 4 - PROGRESS: at 92.45% examples, 66241 words/s, in_qsize 15, out_qsize 0
2019-11-12 11:50:55,972 : INFO : EPOCH 4 - PROGRESS: at 93.15% examples, 66214 words/s, in_qsize 14, out_qsize 1
2019-11-12 11:50:57,096 : INFO : EPOCH 4 - PROGRESS: at 93.97% examples, 66246 words/s, in_qsize 15, out_qsize 0
2019-11-12 11:50:58,138 : INFO : EPOCH 4 - PROGRESS: at 94.50% examples, 66213 words/s, in_qsize 16, out_qsize 0
2019-11-12 11:50:59,230 : INFO : EPOCH 4 - PROGRESS: at 95.10% examples, 66153 words/s, in_qsize 15, out_qsize 0
2019-11-12 11:51:00,254 : INFO : EPOCH 4 - PROGRESS: at 95.53% examples, 66022 words/s, in_qsize 16, out_qsize 0
2019-11-12 11:51:01,326 : INFO : EPOCH 4 - PROGRESS: at 96.12% examples, 65979 words/s, in_qsize

2019-11-12 11:52:07,302 : INFO : EPOCH 5 - PROGRESS: at 48.99% examples, 63167 words/s, in_qsize 15, out_qsize 0
2019-11-12 11:52:08,415 : INFO : EPOCH 5 - PROGRESS: at 49.45% examples, 62988 words/s, in_qsize 15, out_qsize 0
2019-11-12 11:52:09,478 : INFO : EPOCH 5 - PROGRESS: at 50.08% examples, 63378 words/s, in_qsize 15, out_qsize 0
2019-11-12 11:52:10,499 : INFO : EPOCH 5 - PROGRESS: at 50.58% examples, 63286 words/s, in_qsize 16, out_qsize 0
2019-11-12 11:52:11,568 : INFO : EPOCH 5 - PROGRESS: at 51.31% examples, 63160 words/s, in_qsize 15, out_qsize 0
2019-11-12 11:52:12,680 : INFO : EPOCH 5 - PROGRESS: at 52.36% examples, 63512 words/s, in_qsize 15, out_qsize 0
2019-11-12 11:52:13,908 : INFO : EPOCH 5 - PROGRESS: at 53.23% examples, 63345 words/s, in_qsize 15, out_qsize 0
2019-11-12 11:52:14,995 : INFO : EPOCH 5 - PROGRESS: at 54.31% examples, 63664 words/s, in_qsize 15, out_qsize 0
2019-11-12 11:52:16,164 : INFO : EPOCH 5 - PROGRESS: at 55.11% examples, 63515 words/s, in_qsize

(40691969, 61784180)

In [15]:
print("\n\n[INFO] Save the model")
embedding_model_ocr.save("./w2v_005_embedding_model_ocr.model")

2019-11-12 11:53:14,826 : INFO : saving Word2Vec object under ./w2v_005_embedding_model_ocr.model, separately None
2019-11-12 11:53:14,842 : INFO : storing np array 'vectors' to ./w2v_005_embedding_model_ocr.model.wv.vectors.npy




[INFO] Save the model


2019-11-12 11:53:17,473 : INFO : not storing attribute vectors_norm
2019-11-12 11:53:17,474 : INFO : storing np array 'syn1neg' to ./w2v_005_embedding_model_ocr.model.trainables.syn1neg.npy
2019-11-12 11:53:19,409 : INFO : not storing attribute cum_table
2019-11-12 11:53:21,387 : INFO : saved ./w2v_005_embedding_model_ocr.model


## Corrected

In [19]:
list_sentences = db_sentence["corrected_sentencizer_cleaned"].to_list()
print('#sentences: {}'.format(len(list_sentences)))

#sentences: 30509


In [20]:
flattened_list_sentences = [val for sublist in list_sentences for val in sublist]

In [21]:
flattened_list_sentences[0]

['from',
 'river',
 'crossing',
 'to',
 'end',
 'of',
 'trial',
 'splash',
 ':',
 'peler',
 'antill',
 'ploughed',
 'deep',
 'into',
 'paddy',
 "'s",
 'river',
 'in',
 'his',
 'chrysler',
 'plymouth',
 'during',
 'the',
 'elimination',
 'section',
 '.']

In [22]:
embedding_model_corrected = copy.deepcopy(embedding_model_orig)

In [23]:
embedding_model_corrected.workers = 8

In [24]:
embedding_model_corrected.build_vocab(flattened_list_sentences, update=True)
embedding_model_corrected.train(flattened_list_sentences, 
                                total_examples=embedding_model_corrected.corpus_count,
                                epochs=w2v_args.epochs,  
                                compute_loss=w2v_args.compute_loss)

2019-11-12 11:56:12,097 : INFO : collecting all words and their counts
2019-11-12 11:56:12,099 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2019-11-12 11:56:12,254 : INFO : PROGRESS: at sentence #10000, processed 238920 words, keeping 21363 word types
2019-11-12 11:56:12,384 : INFO : PROGRESS: at sentence #20000, processed 462924 words, keeping 30432 word types
2019-11-12 11:56:12,566 : INFO : PROGRESS: at sentence #30000, processed 706176 words, keeping 39228 word types
2019-11-12 11:56:12,770 : INFO : PROGRESS: at sentence #40000, processed 967747 words, keeping 46390 word types
2019-11-12 11:56:12,951 : INFO : PROGRESS: at sentence #50000, processed 1206328 words, keeping 52319 word types
2019-11-12 11:56:13,184 : INFO : PROGRESS: at sentence #60000, processed 1454073 words, keeping 57131 word types
2019-11-12 11:56:13,351 : INFO : PROGRESS: at sentence #70000, processed 1709317 words, keeping 61561 word types
2019-11-12 11:56:13,529 : INFO : PROGRESS: 

2019-11-12 11:56:41,903 : INFO : EPOCH 1 - PROGRESS: at 13.38% examples, 64998 words/s, in_qsize 15, out_qsize 0
2019-11-12 11:56:42,919 : INFO : EPOCH 1 - PROGRESS: at 14.30% examples, 66210 words/s, in_qsize 15, out_qsize 0
2019-11-12 11:56:44,041 : INFO : EPOCH 1 - PROGRESS: at 15.15% examples, 65797 words/s, in_qsize 15, out_qsize 0
2019-11-12 11:56:45,200 : INFO : EPOCH 1 - PROGRESS: at 15.89% examples, 64935 words/s, in_qsize 15, out_qsize 0
2019-11-12 11:56:46,329 : INFO : EPOCH 1 - PROGRESS: at 17.15% examples, 66217 words/s, in_qsize 15, out_qsize 0
2019-11-12 11:56:47,453 : INFO : EPOCH 1 - PROGRESS: at 17.88% examples, 65583 words/s, in_qsize 15, out_qsize 0
2019-11-12 11:56:48,473 : INFO : EPOCH 1 - PROGRESS: at 19.13% examples, 66943 words/s, in_qsize 15, out_qsize 0
2019-11-12 11:56:49,613 : INFO : EPOCH 1 - PROGRESS: at 19.80% examples, 66297 words/s, in_qsize 15, out_qsize 0
2019-11-12 11:56:50,662 : INFO : EPOCH 1 - PROGRESS: at 20.84% examples, 67223 words/s, in_qsize

2019-11-12 11:58:00,367 : INFO : EPOCH 1 - PROGRESS: at 78.64% examples, 67129 words/s, in_qsize 15, out_qsize 0
2019-11-12 11:58:01,505 : INFO : EPOCH 1 - PROGRESS: at 79.44% examples, 67106 words/s, in_qsize 15, out_qsize 0
2019-11-12 11:58:02,656 : INFO : EPOCH 1 - PROGRESS: at 80.57% examples, 67209 words/s, in_qsize 15, out_qsize 0
2019-11-12 11:58:03,709 : INFO : EPOCH 1 - PROGRESS: at 81.57% examples, 67246 words/s, in_qsize 15, out_qsize 0
2019-11-12 11:58:04,864 : INFO : EPOCH 1 - PROGRESS: at 82.70% examples, 67340 words/s, in_qsize 15, out_qsize 0
2019-11-12 11:58:05,893 : INFO : EPOCH 1 - PROGRESS: at 83.64% examples, 67373 words/s, in_qsize 15, out_qsize 0
2019-11-12 11:58:06,915 : INFO : EPOCH 1 - PROGRESS: at 84.62% examples, 67491 words/s, in_qsize 16, out_qsize 0
2019-11-12 11:58:07,930 : INFO : EPOCH 1 - PROGRESS: at 85.57% examples, 67548 words/s, in_qsize 15, out_qsize 0
2019-11-12 11:58:09,013 : INFO : EPOCH 1 - PROGRESS: at 86.41% examples, 67492 words/s, in_qsize

2019-11-12 11:59:14,938 : INFO : EPOCH 2 - PROGRESS: at 42.01% examples, 69743 words/s, in_qsize 16, out_qsize 0
2019-11-12 11:59:16,030 : INFO : EPOCH 2 - PROGRESS: at 43.00% examples, 69951 words/s, in_qsize 15, out_qsize 0
2019-11-12 11:59:17,051 : INFO : EPOCH 2 - PROGRESS: at 43.76% examples, 69866 words/s, in_qsize 15, out_qsize 0
2019-11-12 11:59:18,112 : INFO : EPOCH 2 - PROGRESS: at 44.65% examples, 69855 words/s, in_qsize 15, out_qsize 0
2019-11-12 11:59:19,253 : INFO : EPOCH 2 - PROGRESS: at 45.79% examples, 70001 words/s, in_qsize 15, out_qsize 0
2019-11-12 11:59:20,272 : INFO : EPOCH 2 - PROGRESS: at 46.80% examples, 70033 words/s, in_qsize 15, out_qsize 0
2019-11-12 11:59:21,310 : INFO : EPOCH 2 - PROGRESS: at 47.78% examples, 70094 words/s, in_qsize 15, out_qsize 0
2019-11-12 11:59:22,335 : INFO : EPOCH 2 - PROGRESS: at 48.72% examples, 70049 words/s, in_qsize 15, out_qsize 0
2019-11-12 11:59:23,358 : INFO : EPOCH 2 - PROGRESS: at 49.67% examples, 70007 words/s, in_qsize

2019-11-12 12:00:27,879 : INFO : EPOCH 3 - PROGRESS: at 6.19% examples, 68449 words/s, in_qsize 15, out_qsize 0
2019-11-12 12:00:28,928 : INFO : EPOCH 3 - PROGRESS: at 7.32% examples, 71865 words/s, in_qsize 15, out_qsize 0
2019-11-12 12:00:30,120 : INFO : EPOCH 3 - PROGRESS: at 8.01% examples, 69202 words/s, in_qsize 16, out_qsize 0
2019-11-12 12:00:31,171 : INFO : EPOCH 3 - PROGRESS: at 9.25% examples, 71783 words/s, in_qsize 15, out_qsize 0
2019-11-12 12:00:32,356 : INFO : EPOCH 3 - PROGRESS: at 10.02% examples, 69626 words/s, in_qsize 15, out_qsize 0
2019-11-12 12:00:33,371 : INFO : EPOCH 3 - PROGRESS: at 11.24% examples, 71952 words/s, in_qsize 16, out_qsize 0
2019-11-12 12:00:34,758 : INFO : EPOCH 3 - PROGRESS: at 11.96% examples, 69170 words/s, in_qsize 15, out_qsize 0
2019-11-12 12:00:36,165 : INFO : EPOCH 3 - PROGRESS: at 13.27% examples, 69766 words/s, in_qsize 15, out_qsize 0
2019-11-12 12:00:37,212 : INFO : EPOCH 3 - PROGRESS: at 13.99% examples, 69434 words/s, in_qsize 15,

...

2019-11-12 12:03:04,951 : INFO : EPOCH 4 - PROGRESS: at 37.32% examples, 60347 words/s, in_qsize 15, out_qsize 0
2019-11-12 12:03:06,139 : INFO : EPOCH 4 - PROGRESS: at 37.94% examples, 59992 words/s, in_qsize 16, out_qsize 0
2019-11-12 12:03:07,244 : INFO : EPOCH 4 - PROGRESS: at 38.57% examples, 59740 words/s, in_qsize 15, out_qsize 0
2019-11-12 12:03:08,256 : INFO : EPOCH 4 - PROGRESS: at 39.28% examples, 59583 words/s, in_qsize 15, out_qsize 0
2019-11-12 12:03:09,385 : INFO : EPOCH 4 - PROGRESS: at 39.91% examples, 59335 words/s, in_qsize 15, out_qsize 0
2019-11-12 12:03:10,569 : INFO : EPOCH 4 - PROGRESS: at 40.57% examples, 59042 words/s, in_qsize 16, out_qsize 0
2019-11-12 12:03:11,586 : INFO : EPOCH 4 - PROGRESS: at 41.14% examples, 58814 words/s, in_qsize 16, out_qsize 0
2019-11-12 12:03:12,714 : INFO : EPOCH 4 - PROGRESS: at 41.69% examples, 58487 words/s, in_qsize 15, out_qsize 0
2019-11-12 12:03:13,788 : INFO : EPOCH 4 - PROGRESS: at 42.23% examples, 58226 words/s, in_qsize

2019-11-12 12:04:27,409 : INFO : EPOCH 4 - PROGRESS: at 84.96% examples, 51799 words/s, in_qsize 15, out_qsize 0
2019-11-12 12:04:28,524 : INFO : EPOCH 4 - PROGRESS: at 85.67% examples, 51773 words/s, in_qsize 15, out_qsize 0
2019-11-12 12:04:29,624 : INFO : EPOCH 4 - PROGRESS: at 86.22% examples, 51700 words/s, in_qsize 15, out_qsize 0
2019-11-12 12:04:30,728 : INFO : EPOCH 4 - PROGRESS: at 86.55% examples, 51480 words/s, in_qsize 16, out_qsize 0
2019-11-12 12:04:31,769 : INFO : EPOCH 4 - PROGRESS: at 87.05% examples, 51382 words/s, in_qsize 16, out_qsize 0
2019-11-12 12:04:32,917 : INFO : EPOCH 4 - PROGRESS: at 87.63% examples, 51293 words/s, in_qsize 15, out_qsize 0
2019-11-12 12:04:33,959 : INFO : EPOCH 4 - PROGRESS: at 88.15% examples, 51190 words/s, in_qsize 15, out_qsize 0
2019-11-12 12:04:35,034 : INFO : EPOCH 4 - PROGRESS: at 88.61% examples, 51084 words/s, in_qsize 15, out_qsize 0
2019-11-12 12:04:36,121 : INFO : EPOCH 4 - PROGRESS: at 89.10% examples, 51025 words/s, in_qsize

2019-11-12 12:05:45,800 : INFO : EPOCH 5 - PROGRESS: at 24.08% examples, 42867 words/s, in_qsize 15, out_qsize 0
2019-11-12 12:05:46,875 : INFO : EPOCH 5 - PROGRESS: at 24.78% examples, 43012 words/s, in_qsize 16, out_qsize 0
2019-11-12 12:05:48,175 : INFO : EPOCH 5 - PROGRESS: at 25.49% examples, 42947 words/s, in_qsize 15, out_qsize 0
2019-11-12 12:05:49,303 : INFO : EPOCH 5 - PROGRESS: at 26.16% examples, 43058 words/s, in_qsize 15, out_qsize 0
2019-11-12 12:05:50,734 : INFO : EPOCH 5 - PROGRESS: at 26.81% examples, 42896 words/s, in_qsize 15, out_qsize 0
2019-11-12 12:05:52,153 : INFO : EPOCH 5 - PROGRESS: at 27.46% examples, 42756 words/s, in_qsize 16, out_qsize 0
2019-11-12 12:05:53,565 : INFO : EPOCH 5 - PROGRESS: at 28.07% examples, 42630 words/s, in_qsize 15, out_qsize 1
2019-11-12 12:05:54,653 : INFO : EPOCH 5 - PROGRESS: at 28.76% examples, 42750 words/s, in_qsize 15, out_qsize 0
2019-11-12 12:05:55,786 : INFO : EPOCH 5 - PROGRESS: at 29.39% examples, 42841 words/s, in_qsize

2019-11-12 12:07:09,979 : INFO : EPOCH 5 - PROGRESS: at 67.85% examples, 42154 words/s, in_qsize 15, out_qsize 0
2019-11-12 12:07:11,279 : INFO : EPOCH 5 - PROGRESS: at 68.45% examples, 42148 words/s, in_qsize 15, out_qsize 0
2019-11-12 12:07:12,613 : INFO : EPOCH 5 - PROGRESS: at 69.03% examples, 42134 words/s, in_qsize 16, out_qsize 0
2019-11-12 12:07:14,002 : INFO : EPOCH 5 - PROGRESS: at 69.64% examples, 42100 words/s, in_qsize 15, out_qsize 0
2019-11-12 12:07:15,606 : INFO : EPOCH 5 - PROGRESS: at 70.31% examples, 42000 words/s, in_qsize 16, out_qsize 0
2019-11-12 12:07:16,607 : INFO : EPOCH 5 - PROGRESS: at 70.88% examples, 42032 words/s, in_qsize 16, out_qsize 0
2019-11-12 12:07:17,855 : INFO : EPOCH 5 - PROGRESS: at 71.21% examples, 41849 words/s, in_qsize 15, out_qsize 1
2019-11-12 12:07:19,086 : INFO : EPOCH 5 - PROGRESS: at 71.91% examples, 41866 words/s, in_qsize 15, out_qsize 0
2019-11-12 12:07:20,269 : INFO : EPOCH 5 - PROGRESS: at 72.63% examples, 41898 words/s, in_qsize

(40845481, 60620435)

In [25]:
print("\n\n[INFO] Save the model")
embedding_model_ocr.save("./w2v_005_embedding_model_corrected.model")

2019-11-12 12:08:11,907 : INFO : saving Word2Vec object under ./w2v_005_embedding_model_corrected.model, separately None
2019-11-12 12:08:11,911 : INFO : storing np array 'vectors' to ./w2v_005_embedding_model_corrected.model.wv.vectors.npy




[INFO] Save the model


2019-11-12 12:08:17,457 : INFO : not storing attribute vectors_norm
2019-11-12 12:08:17,462 : INFO : storing np array 'syn1neg' to ./w2v_005_embedding_model_corrected.model.trainables.syn1neg.npy
2019-11-12 12:08:22,634 : INFO : not storing attribute cum_table
2019-11-12 12:08:26,689 : INFO : saved ./w2v_005_embedding_model_corrected.model
